In [37]:
%matplotlib inline
from tqdm import tqdm
import numpy as np
import pymc3 as pm

In [38]:
# Parameters
N = 100 # True total individuals
S = 100  # Sample periods

# Arrival: 40% initially present
birth = np.concatenate(([.4], np.ones(S-1) * (1-.4)/(S-1)), axis=0) 

# Departure: fixed 20% rate All depart at 
dep  = np.concatenate((np.ones(S-1) * .2, [1.]), axis=0)

# Capture probability (propensity)
alpha = .25
beta = 5
prop = pm.Beta.dist(alpha, beta).random(size = N)

In [39]:
# Capture History Data Structures
CH = [] # Full CH matrix
GCH = [] # First/Last/Count CH matrix

# Generate and Arrival Distribution lmd ~ Cat(arr)
Arrival_DIST = pm.Categorical.dist(birth)

for i in tqdm(range(N)):
    arrival = Arrival_DIST.random()
    dep_tmp = dep.copy()
    dep_tmp[:arrival] = 0
    dep_tmp = np.append([1],(1-dep_tmp).cumprod())[:-1] * dep_tmp
    departure = pm.Categorical.dist(dep_tmp).random() 
    dur = departure - arrival + 1
    capture_history = pm.Bernoulli.dist(prop[i]).random(size=(dur,1))
    capture_history = np.concatenate((np.zeros(arrival), 
                                      np.array(capture_history), 
                                      np.zeros(S - departure - 1)), 
                                     axis = 0)
    if capture_history.any():
        first = capture_history.argmax()
        last = S - capture_history[::-1].argmax() -1
        count = capture_history.sum()
        GCH.append([first, last, count])
        CH.append(capture_history)

100%|██████████| 100/100 [00:12<00:00,  8.20it/s]


In [40]:
import pickle
fname = 'data/gch-%d-%d-%.2f-%.2f.pkl' % (N, S, alpha, beta)
fname
with open(fname, 'wb') as f:
  pickle.dump(GCH, f)

In [41]:
fname

'data/gch-100-100-0.25-5.00.pkl'